In [1]:
import synth

In [2]:
s = synth.Synth(params={
        'lfo1_type': 'sine', 
        'lfo1_freq': 0.5, 
        'lfo1_depth': 0.5, 
        'lfo2_type': 'sine', 
        'lfo2_freq': 0.5, 
        'lfo2_depth': 0.5, 
        'osc1_freq': 200.0, 
        'osc1_relative_shift': 1.0, 
        'osc1_wave': 'square', 
        'osc1_detune': 30.0, 
        'osc2_freq': 400.0, 
        'osc2_relative_shift': 1.0, 
        'osc2_wave': 'triangle', 
        'osc2_detune': 30.0, 
        'n_voices': 6, 
        'sub_freq': 100.0, 
        'sub_wave': 'sine', 
        'sub_detune': 30.0, 
        'hf_noise_freq': 1000.0, 
        'hf_noise_type': 'sine', # TODO FIX THIS PROCESSING FOR THE INTERPOLATION
        'lf_noise_freq': 100.0, 
        'lf_noise_type': 'sine', # TODO FIX THIS PROCESSING FOR THE INTERPOLATION
        'filter1_type': 'lowpass', 
        'filter1_freq': 1000.0, 
        'filter2_type': 'na', 
        'filter2_freq': 0.0, 
        'filter3_type': 'na', 
        'filter3_freq': 0.0, 
        'reverb': 0.5, 
        'vibrato_freq': 0.5, 
        'vibrato_depth': 0.5, 
        'attack': 500.0, 
        'decay': 0.5, 
        'sustain': 0.5, 
        'release': 500.0, 
        'fade_in': 0.5, 
        'fade_out': 0.5})

In [1]:
s.generate_audio(69, 2e3).play()

NameError: name 's' is not defined

In [5]:
param = 'hf_noise_type'
param.find('noisea')

-1

In [1]:
import main

In [2]:
ams = main.AmbientMusicSynthesis()
audio = ams.generate_audio(90, {0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1})

Debug | time_interval: (0.0, 20.3)
Debug | next_emotion_segment: 2
Debug | next_emotion_segment: 2
Debug | next_emotion_segment: 2
Debug | next_emotion_segment: 2
Debug | next_emotion_segment: 2
Debug | '------------------------------------------'
Debug | len(time_interval_chords): 12
Debug | len(time_interval_durations): 12
Debug | len(time_interval_durations) == len(time_interval_chords): True
Debug | len(time_interval_chords): 12
Debug | len(time_interval_durations): 12
Debug | len(chords): 12
Debug | len(durations): 12
Debug | time_interval: (20.3, 40.6)
Debug | '------------------------------------------'
Debug | len(time_interval_chords): 7
Debug | len(time_interval_durations): 7
Debug | len(time_interval_durations) == len(time_interval_chords): True
Debug | len(time_interval_chords): 7
Debug | len(time_interval_durations): 7
Debug | len(chords): 19
Debug | len(durations): 19
Debug | time_interval: (40.6, 60.9)
Debug | '------------------------------------------'
Debug | len(time

In [4]:
ams.s.osc1.play()

ValueError: operands could not be broadcast together with shapes (2,) (98342,) 

In [7]:
import gensound
def detuned_array(pitch, duration, width, amount):
  # amount = how many oscillators in the array
  # width = the difference in cents between the highest and lowest oscillators in the array
  all_cents = [i*width/amount - width/2 for i in range(amount)] # how much to detune each signal in the array
  return gensound.mix([ gensound.signals.Square(f"{pitch}{round(cents):+}", duration) for cents in all_cents ])

detuned_array("D4", duration=0.5e3, width=0, amount=1).play() # just one

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
audio.play()

ValueError: operands could not be broadcast together with shapes (4,) (274743,) 

In [31]:
import numpy as np
import gensound 
from gensound.signals import Sine, Sawtooth, Square, Triangle, WhiteNoise, Silence, PinkNoise, Mix, WAV
from gensound.effects import OneImpulseReverb, Vibrato
from gensound.filters import SimpleBandPass, SimpleHPF, SimpleLPF, SimpleHighShelf, SimpleLowShelf
from gensound.transforms import ADSR, Fade, Amplitude, CrossFade
from gensound.curve import SineCurve, Constant, Line

def midi_to_freq(midi_number):
    return 440 * 2**((midi_number-69)/12)
def shift_freq_by_cents(freq, cents):
        return freq*(10**(cents/(1200*3.322038403)))
def detuned_voices(midi_number, relative_shift, duration, detune_pct, n_voices, wave_type='square'):
    # n_voices = how many oscillators in the array
    # detune_range = the difference in cents between the highest and lowest oscillators in the array
    freq = midi_to_freq(midi_number)#self.pitch_to_freq(self.NoteToMidi(pitch))
    # ic(freq)
    detune_range = freq * int(detune_pct)/100
    all_cents = [i*detune_range/n_voices - detune_range/2 for i in range(n_voices)] # how much to detune each signal in the array
    # print(isinstance(pitch, str))
    match wave_type: # REQUIRES PYTHON 3.10
        case 'sine':
            f = Sine
        case 'square':
            f = Square
        case 'triangle':
            f = Triangle
        case 'sawtooth':
            f = Sawtooth
    return gensound.mix([f(shift_freq_by_cents(freq*relative_shift,cents),duration) for cents in all_cents])
def generate_audio(midi, duration):
    lfo1 = Amplitude(SineCurve(frequency=16, depth=0.05, baseline=0.5, duration=duration))
    lfo2 = Amplitude(SineCurve(frequency=0.5, depth=0.5, baseline=0.5, duration=duration))

    osc1 = detuned_voices(midi, 1, duration, 30, 6, 'square')
    osc2 = detuned_voices(midi, 1, duration, 30, 6, 'triangle')
    output = gensound.mix([osc1*lfo1,osc2*lfo2])
    output *= SimpleLPF(1000)
    output*= SimpleHPF(100)
    output *= ADSR(attack=0.5, decay=0.5, sustain=0.9, release=2)

    return output

chords = np.array([np.array([65,60]), np.array([70,74,77]), np.array([39,46]), np.array([44,48,51,56,63])])
durations = [5, 6, 2, 7]
for i in range(len(chords)):
    audio = audio | CrossFade(duration=0.5*1e3) | gensound.mix([generate_audio(midi, durations[i]*1e3) for midi in chords[i]])

audio.play()

In [18]:
chords = np.array([np.array([65,72]), np.array([70,74,77]), np.array([39,46]), np.array([44,48,51,56,63])])
durations = [5, 6, 2, 7]
for i in range(len(chords)):
    if i == 0:
        audio = gensound.mix([generate_audio(midi, durations[i]) for midi in chords[i]])
    else:
        audio = audio | gensound.mix([generate_audio(midi, durations[i]) for midi in chords[i]])

In [26]:
audio.play()

In [21]:
audio.play()